### 1. Using HuggingFace Transformers

In [ ]:
!pip show transformers

In [ ]:
from transformers import AutoModel, AutoTokenizer
from sklearn.preprocessing import normalize

model = AutoModel.from_pretrained('infgrad/stella-base-zh-v2')
tokenizer = AutoTokenizer.from_pretrained('infgrad/stella-base-zh-v2')
sentences = ["数据1", "数据ABCDEFGH"]
batch_data = tokenizer(
    batch_text_or_text_pairs=sentences,
    padding="longest",
    return_tensors="pt",
    max_length=1024,
    truncation=True,
)
attention_mask = batch_data["attention_mask"]
model_output = model(**batch_data)
last_hidden = model_output.last_hidden_state.masked_fill(~attention_mask[..., None].bool(), 0.0)
vectors = last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]
vectors = normalize(vectors, norm="l2", axis=1, )
print(vectors.shape)  # 2,768


In [1]:
from transformers import AutoTokenizer, AutoModel
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'--device={device}')


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-large-zh-v1.5')
tokenizer.padding_side = 'right'
model = AutoModel.from_pretrained('BAAI/bge-large-zh-v1.5').half()

model.to(device) 
model.requires_grad_(False)
model.eval()

--device=cuda:0
[2023-11-24 05:35:14,159] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inpl

In [2]:
sentences = ['你分享的信息显示你想查看语法"ALTER SYSTEM SET SMTP_OUT_SERVER =…"的文档？  从RDS Oracle公共文档进一步研究，我能找到：[+]<https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/Oracle.Options.UTLMAIL.html>  SMTP_OUT_SERVER是一个客户可以设置的参数。为了更改系统，客户需要修改该参数。  "您可以使用DB参数组为DB实例设置默认SMTP_OUT_SERVER。您可以通过在DB实例上运行以下代码来为会话设置SMTP_OUT_SERVER参数。   ALTER SESSION SET smtp_out_server = mailserver.domain.com:25 ;"  希望以上内容有所帮助。', 'Title:有关Oracle"ALTER SYSTEM SET SMTP_OUT_SERVER =…"的rdsadmin语法请求帮助。 Content:这似乎是那些没有被文档化为"通用示例"的特定命令之一，当然我们无法查看rdsadmin_util软件包中的过程。至于邮件服务器，它的解析源是URL而不是IP地址。谢谢。 Answer:你分享的信息显示你想查看语法"ALTER SYSTEM SET SMTP_OUT_SERVER =…"的文档？  从RDS Oracle公共文档进一步研究，我能找到：[+]<https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/Oracle.Options.UTLMAIL.html>  SMTP_OUT_SERVER是一个客户可以设置的参数。为了更改系统，客户需要修改该参数。  "您可以使用DB参数组为DB实例设置默认SMTP_OUT_SERVER。您可以通过在DB实例上运行以下代码来为会话设置SMTP_OUT_SERVER参数。   ALTER SESSION SET smtp_out_server = mailserve', '在本地环境中，最佳的处理数据库的方式是什么？ ', '我们目前正在使用Aurora RDS postgres数据库。最初，我们的数据库很小，容易管理，我们将转储发送到S3并提供给开发人员开发新功能和解决错误。  目前我们的数据库重达700GB，**最好的策略是什么，以便开发人员可以获得数据？**下载整个数据库是不可能的，这将浪费大量时间成本。最重要的是，开发人员能够获得数据库的副本来构建开发环境，以支持他们开发新功能和解决缺陷。 ', '首先，考虑一下是否拥有一个所有开发人员都连接的单个副本可以满足您的需求。他们可以远程连接，并且它将是您生产数据的一个副本。因此，它非常棒，因为很容易定期更新并保证覆盖所有生产用例。它有很多优势。  但这可能不能满足您的需求。您说您需要本地数据库。在这种情况下，我认为您有两条主要路径要考虑。  1. 生成示例数据 2. 创建提取生产数据子集的过程  在开始时，生成示例数据可能需要更多的工作。但它有一些不错的优点。很容易确保您生成所需的数据。它将是参数化的，因此每个开发人员在那个时刻生成他/她关心的数据。下载大数据集没有网络问题。  但如果您确实需要提取主数据库的一部分，那么您需要将其视为提取-转换-加载(ETL)项目。使用数据集成(DI/ETL)工具连接到主数据库并提取一些子集。理想情况下，子集将很容易定义。也许对于大多数表，您只需取最近2个月的数据，对于其他表(如参考表)，则取整个表。定义所有单个映射可能需要很大的努力...但实际上并不复杂。您可以决定详细信息，如将数据加载到另一个数据库或保存到CSV文件中。然后使数据库转储或CSV文件可供您的开发人员使用。作为一名开发人员，您可能会倾向于编写自己的脚本来执行']

In [5]:
sentences = sentences + sentences
len(sentences)

10

In [6]:
cnt = 0 
for _ in range(600):
    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt').to(device)
    # for s2p(short query to long passage) retrieval task, add an instruction to query (not add instruction for passages)
    # encoded_input = tokenizer([instruction + q for q in queries], padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)
        sentence_embeddings = model_output[0][:, 0]
        # Perform pooling. In this case, cls pooling

    # Perform pooling. In this case, max pooling.
    sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1).cpu().numpy()
    cnt = cnt + 1
    if cnt % 100 == 0:
        print(f"finsih test is {cnt}")
    # print("Sentence embeddings:", sentence_embeddings)

finsih test is 100
finsih test is 200
finsih test is 300
finsih test is 400
finsih test is 500
finsih test is 600


In [6]:
len(encoded_input.input_ids[0])

512

In [ ]:
for sent in sentences:
    # Tokenize sentences
    encoded_input = tokenizer(sent[:512], padding=True, truncation=True, return_tensors='pt').to(device)
    # for s2p(short query to long passage) retrieval task, add an instruction to query (not add instruction for passages)
    # encoded_input = tokenizer([instruction + q for q in queries], padding=True, truncation=True, return_tensors='pt')
    
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)
        sentence_embeddings = model_output[0][:, 0]
        # Perform pooling. In this case, cls pooling
    
    # Perform pooling. In this case, max pooling.
    sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1).cpu().numpy()
    print("Sentence embeddings:", sentence_embeddings)

In [ ]:
# cross model